In [1]:
from tqdm import tqdm
import h5py 
import os
import re
import subprocess
import pandas as pd

In [2]:
folder_path = "../"
write_path = "../gwtc3_converted/"

PLANCK_2018_Ho = 2.1816926176539463e-18 ### CGS
PLANCK_2018_OmegaMatter = 0.3158
PLANCK_2018_OmegaLambda = 1. - PLANCK_2018_OmegaMatter
PLANCK_2018_OmegaRadiation = 0.

In [3]:
# Read the data
files = (file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file)))
events_files = list(filter(lambda x: x.endswith((".h5", ".hdf5")), files))
for i in range(len(events_files)):print(f"{i+1}. {events_files[i]} (i = {i})")

1. IGWN-GWTC3p0-v1-GW200225_060421_PEDataRelease_mixed_cosmo.h5 (i = 0)
2. IGWN-GWTC2p1-v2-GW190521_074359_PEDataRelease_mixed_cosmo.h5 (i = 1)
3. IGWN-GWTC2p1-v2-GW190915_235702_PEDataRelease_mixed_cosmo.h5 (i = 2)
4. IGWN-GWTC3p0-v1-GW200224_222234_PEDataRelease_mixed_cosmo.h5 (i = 3)
5. IGWN-GWTC2p1-v2-GW190706_222641_PEDataRelease_mixed_cosmo.h5 (i = 4)
6. IGWN-GWTC2p1-v2-GW190412_053044_PEDataRelease_mixed_cosmo.h5 (i = 5)
7. IGWN-GWTC2p1-v2-GW190929_012149_PEDataRelease_mixed_cosmo.h5 (i = 6)
8. IGWN-GWTC3p0-v1-GW200209_085452_PEDataRelease_mixed_cosmo.h5 (i = 7)
9. IGWN-GWTC3p0-v1-GW200219_094415_PEDataRelease_mixed_cosmo.h5 (i = 8)
10. IGWN-GWTC2p1-v2-GW190828_065509_PEDataRelease_mixed_cosmo.h5 (i = 9)
11. IGWN-GWTC3p0-v1-GW191129_134029_PEDataRelease_mixed_cosmo.h5 (i = 10)
12. IGWN-GWTC3p0-v1-GW191109_010717_PEDataRelease_mixed_cosmo.h5 (i = 11)
13. IGWN-GWTC2p1-v2-GW190408_181802_PEDataRelease_mixed_cosmo.h5 (i = 12)
14. IGWN-GWTC2p1-v2-GW190707_093326_PEDataRelease_mixed_c

In [4]:
if not os.path.exists(write_path + "event-list.txt"):
    with open(write_path + "event-list.txt", "w") as f:
        pass

In [5]:
#regex
re1 = r"GWTC-1"
re2= r"GWTC2p1"
re3 = r"GWTC3"

In [6]:
def contains_pattern(s, options_list):
    combined_pattern = "|".join(options_list)
    return bool(re.search(combined_pattern, s))

In [7]:
event_waveforms = []
compile_df = pd.DataFrame(columns=["event", "waveform", "cosmology", "prior_cosmology1", "prior_cosmology2", "prior_a_1 (or chi1)", "prior_a_2 (or chi2)", "prior_tilt_1", "prior_tilt_2"])


for i in tqdm(range(len(events_files))):
    file_path = events_files[i]
    event_file = folder_path+file_path
    with h5py.File(event_file,'r+') as file: 
        wave_index = 0
        if contains_pattern(file_path, ["GW200115"]):
            wave_index = 2
        waveform = list(file.keys())[wave_index]
        # print(file_path)
        # print(file.keys())
        # print(file[waveform].dtype.descr)
        # print(file[waveform]["priors"].keys())
        # print(file[waveform]["meta_data"]['meta_data'].keys())
        # print(file[waveform]["description"][:])
        # print(file[waveform]["priors"]["analytic"].keys())
        # print(file[waveform]["meta_data"]['meta_data'].keys())
        # print(file[waveform]["priors"]['analytic']["luminosity_distance"][:][0])
        # print(file[waveform]["priors"]['analytic']["a_1"][:][0])
        # print(compile_df.columns)
        # print(list_row)
        # print(file[waveform]["posterior_samples"].dtype.descr)
        if contains_pattern(file_path, ["GW190707", "GW190814", "GW170608", "GW190728", "GW190924", "GW190720"]):
            spin1_used = "NOT INCLUDED (Uniform)" ; spin2_used = "NOT INCLUDED (Uniform)"      
            tilt1_used = "NOT INCLUDED (Isotropic)" ; tilt2_used = "NOT INCLUDED (Isotropic)"
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            cosmo_prior = "NOT INCLUDED"
        elif contains_pattern(file_path, ["GW190425"]): # GW190425 - https://arxiv.org/pdf/2108.01045
            spin1_used = "chi < 0.89 (Uniform)"; spin2_used = "chi < 0.89 (Uniform)"   
            tilt1_used = "NOT INCLUDED (Isotropic)" ; tilt2_used = "NOT INCLUDED (Isotropic)"
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            cosmo_prior = "NOT INCLUDED"
        elif contains_pattern(file_path, ["GW170817"]): # GW170817 - https://arxiv.org/pdf/1805.11579
            # print(file["IMRPhenomPv2NRT_highSpin_prior"]["costilt1"])
            # print(file[waveform].dtype.descr)
            cosmology_used = "planck 2015"
            cosmo_prior = "NOT INCLUDED"
            spin1_used = "chi < 0.89 (Uniform)"; spin2_used = "chi < 0.89 (Uniform)"
            tilt1_used = "NOT INCLUDED (Isotropic)"; tilt2_used = "NOT INCLUDED (Isotropic)"
        else:
            cosmo_prior = file[waveform]["priors"]['analytic']["luminosity_distance"][:][0]
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            spin1_used = str(file[waveform]["priors"]['analytic']["a_1"][:][0]) ; spin2_used = str(file[waveform]["priors"]['analytic']["a_2"][:][0])
            tilt1_used = str(file[waveform]["priors"]['analytic']["tilt_1"][:][0]) ; tilt2_used = str(file[waveform]["priors"]['analytic']["tilt_2"][:][0])
            
        list_row = [events_files[i], waveform, cosmology_used, cosmo_prior, spin1_used, spin2_used, tilt1_used, tilt2_used]
        event_waveforms.append(f"Converting {waveform} from {event_file}")
                                               
            
    args = [f"{waveform}", f"{event_file}",
            "--Ho", f"{PLANCK_2018_Ho}", "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", "--OmegaRadiation",
            f"{PLANCK_2018_OmegaRadiation}", "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", "-o", f"{write_path}"]
    
    if re.search(re1, file_path):
        cosmo_cat = "d_l^2"  # According to Amanda's O4a work
        subprocess.run(["python", "h52csv_GWTC1.py", *args])
    elif re.search(re2, file_path):
        cosmo_cat = "uniform in comoving volume"
        subprocess.run(["python", "h52csv_GWTC2p1.py", *args])
    elif re.search(re3, file_path):
        cosmo_cat = "uniform in comoving volume"
        subprocess.run(["python", "h52csv_GWTC3.py", *args])
    else:
        raise ValueError("No match found")
    
    list_row = [events_files[i], waveform, cosmology_used, cosmo_cat, cosmo_prior, spin1_used, spin2_used, tilt1_used, tilt2_used]
    compile_df.loc[i] = list_row
    
compile_df.to_csv("compile_df.csv", index=False)
compile_df

100%|██████████| 66/66 [02:58<00:00,  2.71s/it]


,event,waveform,cosmology,prior_cosmology1,prior_cosmology2,prior_a_1 (or chi1),prior_a_2 (or chi2),prior_tilt_1,prior_tilt_2
0,IGWN-GWTC3p0-v1-GW200225_060421_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(minimum=0, maximum=3.141592653589793, n...","b""Sine(minimum=0, maximum=3.141592653589793, n..."
1,IGWN-GWTC2p1-v2-GW190521_074359_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=10, maximum=10000,...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
2,IGWN-GWTC2p1-v2-GW190915_235702_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=10, maximum=10000,...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(minimum=0, maximum=3.141592653589793, n...","b""Sine(minimum=0, maximum=3.141592653589793, n..."
3,IGWN-GWTC3p0-v1-GW200224_222234_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(minimum=0, maximum=3.141592653589793, n...","b""Sine(minimum=0, maximum=3.141592653589793, n..."
4,IGWN-GWTC2p1-v2-GW190706_222641_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
...,...,...,...,...,...,...,...,...,...
61,IGWN-GWTC2p1-v2-GW190720_000836_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,NOT INCLUDED,NOT INCLUDED (Uniform),NOT INCLUDED (Uniform),NOT INCLUDED (Isotropic),NOT INCLUDED (Isotropic)
62,IGWN-GWTC2p1-v2-GW170729_185629_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
63,IGWN-GWTC3p0-v1-GW200112_155838_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
64,IGWN-GWTC2p1-v2-GW190910_112807_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."


In [8]:
# read txt file
event_txt_files = []
with open(write_path + "event-list.txt", "r") as f:
    event_txt_files = f.readlines()

print("Number of events converted: ", len(event_txt_files))
print("Total number of events: ", len(events_files))

Number of events converted:  66
Total number of events:  66


In [9]:
# Read events and confirm the conversion. 
import numpy as np
event_txt_files = []
with open(write_path + "event-list.txt", "r") as f:
    event_txt_files = f.read().splitlines()
event_txt_files.sort()
event_txt_files

['GW150914_095045.csv.gz',
 'GW151012_095443.csv.gz',
 'GW151226_033853.csv.gz',
 'GW170104_101158.csv.gz',
 'GW170608_020116.csv.gz',
 'GW170729_185629.csv.gz',
 'GW170809_082821.csv.gz',
 'GW170814_103043.csv.gz',
 'GW170817.csv.gz',
 'GW170818_022509.csv.gz',
 'GW170823_131358.csv.gz',
 'GW190408_181802.csv.gz',
 'GW190412_053044.csv.gz',
 'GW190413_134308.csv.gz',
 'GW190421_213856.csv.gz',
 'GW190425_081805.csv.gz',
 'GW190503_185404.csv.gz',
 'GW190512_180714.csv.gz',
 'GW190513_205428.csv.gz',
 'GW190517_055101.csv.gz',
 'GW190519_153544.csv.gz',
 'GW190521_030229.csv.gz',
 'GW190521_074359.csv.gz',
 'GW190527_092055.csv.gz',
 'GW190602_175927.csv.gz',
 'GW190620_030421.csv.gz',
 'GW190630_185205.csv.gz',
 'GW190701_203306.csv.gz',
 'GW190706_222641.csv.gz',
 'GW190707_093326.csv.gz',
 'GW190708_232457.csv.gz',
 'GW190720_000836.csv.gz',
 'GW190727_060333.csv.gz',
 'GW190728_064510.csv.gz',
 'GW190803_022701.csv.gz',
 'GW190814_211039.csv.gz',
 'GW190828_063405.csv.gz',
 'GW1908

In [10]:
import pandas as pd
import matplotlib.pyplot as plt

for i in tqdm(range(len(event_txt_files))):
    event = event_txt_files[i]
    df = pd.DataFrame(np.genfromtxt(write_path + event, delimiter=",", names=True))
    fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(10, 4))
    ax[0, 0].hist(df["mass1_source"]); ax[0, 0].set_xlabel("mass1_source (Msun)")
    ax[0, 1].hist(df["mass2_source"]); ax[0, 1].set_xlabel("mass2_source (Msun)")
    ax[0, 2].hist(df["luminosity_distance"]); ax[0, 2].set_xlabel("luminosity_distance (MPc)")
    ax[0, 3].hist(df["redshift"]); ax[0, 3].set_xlabel("redshift (z)")
    ax[1, 0].hist(df["a_1"]); ax[1, 0].set_xlabel("a_1")
    ax[1, 1].hist(df["a_2"]); ax[1, 1].set_xlabel("a_2")
    ax[1, 2].hist(df["costilt1"]); ax[1, 2].set_xlabel("costilt1")
    ax[1, 3].hist(df["costilt2"]); ax[1, 3].set_xlabel("costilt1")
    plt.suptitle(event[:12] + f" - {df.shape[0]} samples")
    plt.tight_layout()
    plt.savefig("verify_PE/" + event[:12] + ".png")
    plt.close()
    

100%|██████████| 66/66 [00:17<00:00,  3.74it/s]


In [11]:
# i = 48
# event = event_txt_files[i]
# print(event)
# df = pd.DataFrame(np.genfromtxt(write_path + event_txt_files[i], delimiter=",", names=True))
# df_normal = pd.DataFrame(np.genfromtxt(write_path + event_txt_files[50], delimiter=",", names=True))
# print(df.columns)
# print(df_normal.columns) 
#Additionally includes geocenter_time, lnprob_geocenter_time, lnprob_spin1x_spin1y_spin1z, lnprob_spin2x_spin2y_spin2z, loglikelihood, luminosity_distance, spin1x, spin1y, spin1z, spin2x, spin2y, spin2z. (This has changed now but was true before modification of h52csv_GWTC1.py)

# Conversion of Injections

In [12]:
vt_folder = "../../vt/"
vt_file = "o1+o2+o3_mixture_real+semianalytic-LIGO-T2100377-v2.hdf5"
vt_file_O3_full = "endo3_mixture-LIGO-T2100113-v12.hdf5"
vt_path = vt_folder + vt_file

In [13]:
with h5py.File(vt_folder + vt_file_O3_full, "r") as f:
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]
    print(type(f[a_group_key])) 
    data = list(f[a_group_key])
    print(data)
    
with h5py.File(vt_path, "r") as f:
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]
    print(type(f[a_group_key])) 
    data = list(f[a_group_key])
    print(data)

Keys: <KeysViewHDF5 ['injections']>
<class 'h5py._hl.group.Group'>
['declination', 'declination_sampling_pdf', 'detection_statistic_cwb', 'detection_statistic_mbta', 'detection_statistic_pycbc_bbh', 'detection_statistic_pycbc_hyperbank', 'distance', 'far_cwb', 'far_gstlal', 'far_mbta', 'far_pycbc_bbh', 'far_pycbc_hyperbank', 'gps_time', 'gps_time_int', 'ifar_cwb', 'ifar_gstlal', 'ifar_mbta', 'ifar_pycbc_bbh', 'ifar_pycbc_hyperbank', 'inclination', 'inclination_sampling_pdf', 'mass1', 'mass1_source', 'mass2', 'mass2_source', 'mixture_weight', 'optimal_snr_h', 'optimal_snr_l', 'optimal_snr_net', 'pastro_cwb', 'pastro_gstlal', 'pastro_mbta', 'pastro_pycbc_bbh', 'pastro_pycbc_hyperbank', 'polarization', 'redshift', 'right_ascension', 'right_ascension_sampling_pdf', 'sampling_pdf', 'spin1x', 'spin1y', 'spin1z', 'spin2x', 'spin2y', 'spin2z']
Keys: <KeysViewHDF5 ['injections']>
<class 'h5py._hl.group.Group'>
['far_cwb', 'far_gstlal', 'far_mbta', 'far_pycbc_bbh', 'far_pycbc_hyperbank', 'ifar_c

In [14]:
with h5py.File(vt_path,'r+') as file: 
        waveform = list(file.keys())[0]
        event_waveforms.append(f"Converting {waveform} from {vt_path}")
        
args = ["--max-far", f"{1}",
        "--Ho", f"{PLANCK_2018_Ho}", 
        "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", 
        "--OmegaRadiation", f"{PLANCK_2018_OmegaRadiation}", 
        "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", 
        f"{vt_path}",
        # "-v", 
        # f"{vt_folder}" + "sensitivity-estimate_O3.csv.gz"]
        f"{vt_folder}" + "sensitivity-estimate.csv.gz"]

subprocess.run(["python", "h52csv_injections.py", *args])
print("Done")

Done


In [15]:
path = f"{vt_folder}" + "sensitivity-estimate.csv.gz"
df = pd.read_csv(path)
df # Checking injections

,a_1,a_2,costilt1,costilt2,far,geocenter_time,lnprob_geocenter_time,lnprob_mass1_source_mass2_source_redshift_spin1spherical_spin2spherical,mass1_source,mass2_source,redshift,spin1x,spin1y,spin1z,spin2x,spin2y,spin2z
0,0.262990,0.353922,-0.443605,0.123831,2.461024e-05,0.0,1.0,-4.728567,2.149666,1.595966,0.075358,-0.233778,-0.030015,-0.116664,-0.319472,-0.145868,0.043827
1,0.192997,0.283679,-0.969152,-0.408359,1.772536e-05,0.0,1.0,-9.041966,7.257794,2.335832,0.137468,-0.007892,-0.046908,-0.187043,0.214222,-0.145476,-0.115843
2,0.226059,0.229092,0.637536,-0.725247,6.661743e-14,0.0,1.0,-5.480601,2.364014,1.740568,0.050528,-0.170921,-0.033432,0.144120,0.082180,-0.134627,-0.166148
3,0.902902,0.189674,0.265857,-0.723968,1.476506e-05,0.0,1.0,-9.121858,3.141577,1.072391,0.045338,-0.043686,0.869311,0.240043,0.013434,0.130152,-0.137318
4,0.711184,0.794992,0.815303,0.094001,1.526518e-09,0.0,1.0,-18.592321,71.112717,57.015984,0.719476,0.292093,0.290278,0.579830,-0.781097,0.127732,0.074730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221319,0.712716,0.315726,0.854941,0.213853,2.293210e-01,0.0,1.0,-9.912697,6.708300,2.146488,0.076519,-0.257178,0.265595,0.609330,0.103421,0.290565,0.067519
221320,0.203080,0.343764,-0.759649,-0.755698,3.966578e-09,0.0,1.0,-6.907011,2.082222,1.676427,0.024144,0.029651,0.128698,-0.154270,-0.011663,0.224836,-0.259782
221321,0.428977,0.396957,-0.199208,-0.569394,3.807962e-52,0.0,1.0,-12.395740,9.539573,2.426798,0.032087,-0.389226,0.158813,-0.085456,-0.249228,-0.210649,-0.226025
221322,0.631990,0.473451,-0.159549,0.681661,1.050514e-06,0.0,1.0,-12.923044,3.403745,2.128665,0.047793,0.316157,-0.537856,-0.100833,0.245378,0.244518,0.322733


In [82]:
folder_path = "../../events_of_interest/"
write_path = "../../events_of_interest/events_of_interest_converted/"

# Read the data
files = (file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file)))
events_files = list(filter(lambda x: x.endswith((".h5", ".hdf5")), files))
for i in range(len(events_files)):print(f"{i+1}. {events_files[i]} (i = {i})")

if not os.path.exists(write_path + "event-list.txt"):
    with open(write_path + "event-list.txt", "w") as f:
        pass
     
#regex
re1 = r"GWTC-1"
re2= r"GWTC2p1"
re3 = r"GWTC3"
re4 = r"GWTC4"

event_waveforms = []
interest_df = pd.DataFrame(columns=["event", "waveform", "cosmology", "prior_cosmology1", "prior_cosmology2", "prior_a_1 (or chi1)", "prior_a_2 (or chi2)", "prior_tilt_1", "prior_tilt_2"])


for i in tqdm(range(len(events_files))): 
    file_path = events_files[i]
    event_file = folder_path+file_path
    with h5py.File(event_file,'r+') as file: 
        wave_index = 0
        if contains_pattern(file_path, ["GW200115"]):
            wave_index = 2
        elif contains_pattern(file_path, ["GW230529"]):
            wave_index = 9
        elif contains_pattern(file_path, ["GW200105"]):
            wave_index = 1
        waveform = list(file.keys())[wave_index]
        
        if contains_pattern(file_path, ["GW190707", "GW190814", "GW170608", "GW190728", "GW190924", "GW190720", "GW190917"]):
            spin1_used = "NOT INCLUDED (Uniform)" ; spin2_used = "NOT INCLUDED (Uniform)"      
            tilt1_used = "NOT INCLUDED (Isotropic)" ; tilt2_used = "NOT INCLUDED (Isotropic)"
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            cosmo_prior = "NOT INCLUDED"
        elif contains_pattern(file_path, ["GW190425"]): # GW190425 - https://arxiv.org/pdf/2108.01045
            spin1_used = "chi < 0.89 (Uniform)"; spin2_used = "chi < 0.89 (Uniform)"   
            tilt1_used = "NOT INCLUDED (Isotropic)" ; tilt2_used = "NOT INCLUDED (Isotropic)"
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            cosmo_prior = "NOT INCLUDED"
        elif contains_pattern(file_path, ["GW170817"]): # GW170817 - https://arxiv.org/pdf/1805.11579
            # print(file["IMRPhenomPv2NRT_highSpin_prior"]["costilt1"])
            # print(file[waveform].dtype.descr)
            cosmology_used = "planck 2015"
            cosmo_prior = "NOT INCLUDED"
            spin1_used = "chi < 0.89 (Uniform)"; spin2_used = "chi < 0.89 (Uniform)"
            tilt1_used = "NOT INCLUDED (Isotropic)"; tilt2_used = "NOT INCLUDED (Isotropic)"
        elif contains_pattern(file_path, ["GW200105"]):
            spin1_used = "chi < 0.99 (Aligned)" ; spin2_used = "chi < 0.99 (Aligned)"
            tilt1_used = "NOT INCLUDED (Isotropic)" ; tilt2_used = "NOT INCLUDED (Isotropic)"
            cosmology_used = "Planck 2015"
            cosmo_prior = "NOT INCLUDED"
        else:
            cosmo_prior = file[waveform]["priors"]['analytic']["luminosity_distance"][:][0]
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            spin1_used = str(file[waveform]["priors"]['analytic']["a_1"][:][0]) ; spin2_used = str(file[waveform]["priors"]['analytic']["a_2"][:][0])
            tilt1_used = str(file[waveform]["priors"]['analytic']["tilt_1"][:][0]) ; tilt2_used = str(file[waveform]["priors"]['analytic']["tilt_2"][:][0])
            
        list_row = [events_files[i], waveform, cosmology_used, cosmo_prior, spin1_used, spin2_used, tilt1_used, tilt2_used]
        event_waveforms.append(f"Converting {waveform} from {event_file}")
                                               
            
    args = [f"{waveform}", f"{event_file}",
            "--Ho", f"{PLANCK_2018_Ho}", "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", "--OmegaRadiation",
            f"{PLANCK_2018_OmegaRadiation}", "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", "-o", f"{write_path}"]
    
    if re.search(re1, file_path):
        cosmo_cat = "d_l^2"  # According to Amanda's O4a work
        # subprocess.run(["python", "h52csv_GWTC1.py", *args])
    elif re.search(re2, file_path):
        cosmo_cat = "uniform in comoving volume"
        # subprocess.run(["python", "h52csv_GWTC2p1.py", *args])
    elif re.search(re3, file_path):
        cosmo_cat = "uniform in comoving volume"
        # subprocess.run(["python", "h52csv_GWTC3.py", *args])
    elif re.search(re4, file_path):
        cosmo_cat = "uniform in comoving volume"
        # subprocess.run(["python", "h52csv_GWTC3.py", *args])
    else:
        raise ValueError("No match found")
    
    list_row = [events_files[i], waveform, cosmology_used, cosmo_cat, cosmo_prior, spin1_used, spin2_used, tilt1_used, tilt2_used]
    interest_df.loc[i] = list_row
    
interest_df

1. IGWN-GWTC4-manual-download-GW230529.h5 (i = 0)
2. IGWN-GWTC3p0-v1-GW200115_042309_PEDataRelease_mixed_cosmo.h5 (i = 1)
3. IGWN-GWTC2p1-v2-GW190814_211039_PEDataRelease_mixed_cosmo.h5 (i = 2)
4. IGWN-GWTC2p1-v2-GW190917_114630_PEDataRelease_mixed_cosmo.h5 (i = 3)
5. IGWN-GWTC2p1-v2-GW190425_081805_PEDataRelease_mixed_cosmo.h5 (i = 4)
6. IGWN-GWTC3-marginal-manual-download-GW200105.h5 (i = 5)
7. GW170817_GWTC-1.hdf5 (i = 6)


100%|██████████| 7/7 [00:00<00:00, 835.42it/s]


,event,waveform,cosmology,prior_cosmology1,prior_cosmology2,prior_a_1 (or chi1),prior_a_2 (or chi2),prior_tilt_1,prior_tilt_2
0,IGWN-GWTC4-manual-download-GW230529.h5,IMRPhenomXPHM_highSpin,b'Planck15',uniform in comoving volume,"b""bilby.gw.prior.UniformSourceFrame(minimum=1....","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(minimum=0, maximum=3.141592653589793, n...","b""Sine(minimum=0, maximum=3.141592653589793, n..."
1,IGWN-GWTC3p0-v1-GW200115_042309_PEDataRelease_...,C01:IMRPhenomXPHM:HighSpin,b'Planck15_lal',uniform in comoving volume,"b""PowerLaw(alpha=2, minimum=10, maximum=10000,...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(minimum=0, maximum=3.141592653589793, n...","b""Sine(minimum=0, maximum=3.141592653589793, n..."
2,IGWN-GWTC2p1-v2-GW190814_211039_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,NOT INCLUDED,NOT INCLUDED (Uniform),NOT INCLUDED (Uniform),NOT INCLUDED (Isotropic),NOT INCLUDED (Isotropic)
3,IGWN-GWTC2p1-v2-GW190917_114630_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',uniform in comoving volume,NOT INCLUDED,NOT INCLUDED (Uniform),NOT INCLUDED (Uniform),NOT INCLUDED (Isotropic),NOT INCLUDED (Isotropic)
4,IGWN-GWTC2p1-v2-GW190425_081805_PEDataRelease_...,C01:IMRPhenomPv2_NRTidal:HighSpin,b'Planck15_lal',uniform in comoving volume,NOT INCLUDED,chi < 0.89 (Uniform),chi < 0.89 (Uniform),NOT INCLUDED (Isotropic),NOT INCLUDED (Isotropic)
5,IGWN-GWTC3-marginal-manual-download-GW200105.h5,C01:Combined_PHM_high_spin,Planck 2015,uniform in comoving volume,NOT INCLUDED,chi < 0.99 (Uniform),chi < 0.99 (Uniform),NOT INCLUDED (Isotropic),NOT INCLUDED (Isotropic)
6,GW170817_GWTC-1.hdf5,IMRPhenomPv2NRT_highSpin_posterior,planck 2015,d_l^2,NOT INCLUDED,chi < 0.89 (Uniform),chi < 0.89 (Uniform),NOT INCLUDED (Isotropic),NOT INCLUDED (Isotropic)
